**Результаты so far:**
*  Воспроизведен бейзлайн RSG




# Data

In [5]:
%%capture
%%bash
# change url if you want to work with a different RSG dataset
wget -q --show-progress "https://russiansuperglue.com/tasks/download/RuCoS" -O temp.zip
unzip temp.zip -d data

# remove unnecessary directories and files
rm temp.zip
rm -r data/__MACOSX
rm -r sample_data/

In [6]:
# Load necessary code files and models from https://github.com/RussianNLP/RussianSuperGLUE 
# to recreate TfIdf baseline

%%capture
%%bash
# load tfidf pickle created by RSG team
wget -q --show-progress "https://russiansuperglue.com/tasks/tf_idf" -O temp.zip
unzip temp.zip -d data
rm temp.zip

# Make sure you donwload with the raw file link
# Keep the link relevant to your dataset
wget -q --show-progress "https://github.com/RussianNLP/RussianSuperGLUE/raw/master/tfidf_baseline/RuCoS.py" -O RuCoS.py

In [7]:
import pandas as pd
import json
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 80)

class JSONL_handler():
    """ opens a jsonl file and turns it into a necessary data structure """
    
    def __init__(self, path):
        self.path = path # path to jsonl file

    def to_pandas(self):
        """ get jsonl file content as a pandas DataFrame"""

        text_df = pd.DataFrame(columns=['text', 'entities'])
        questions_df = pd.DataFrame(columns=['text_id',
                                             'question', 'answers'])

        lines = self.yield_lines()

        for passage_id, line in enumerate(lines):
            text, entities, questions = self.split_text_and_questions(line)
            text_df = text_df.append({'text':text, 'entities': entities}, 
                           ignore_index=True)
            for i in range(len(questions)):
                questions_df = questions_df.append({'text_id': passage_id,
                                    'question': questions[i]['query'],
                                     'answers': questions[i]['answers']},
                                    ignore_index=True)
        return text_df, questions_df

    def yield_lines(self):
        """ yields json lines one by one """
        with open(self.path) as f:
            for line in f:
                yield json.loads(line)


    def split_text_and_questions(self, line):
        """ transforms a complex json object into a single row dataframe"""
        text = line['passage']['text']
        entities = line['passage']['entities']
        questions = line['qas']

        return text, entities, questions

In [21]:
train = JSONL_handler('data/RuCoS/train.jsonl')
texts_train, questions_train = train.to_pandas()

valid = JSONL_handler('data/RuCoS/val.jsonl')
texts_valid, questions_valid = valid.to_pandas()

In [23]:
texts_train.shape, questions_train.shape, texts_valid.shape, questions_valid.shape

((72193, 2), (72193, 3), (7577, 2), (7577, 3))

In [25]:
texts_train.head()

,text,entities
0,"Наблюдатели полагают, что подоплекой теракта в Домодедово является провал кавказской политики российского правительства, указывает немецкая печать. Немецкая печать продолжает комментировать теракт в Домодедово. Так, газета Süddeutsche Zeitung пишет:\nНу, конечно же, после взрыва в Домодедово вновь обнаруживается ""кавказский след"". Его обнаруживают почти всегда, когда в России взрывается бомба в метро, в поезде или на рынке. И неважно, что очевидцы нередко не в состоянии сказать, был ли преступник женщиной в чадре или же мужчиной, грозившим всех уничтожить.Среди жертв взрыва в Домодедово есть граждане Германии, Великобритании, Австрии. Ежедневно в Москве совершают посадку более 3 десятков самолетов из Германии.\n@highlight\nНемецкий менеджер аэропорта Домодедово отвергает обвинения Медведева\n@highlight\nТеракт в Домодедово: системные просчеты\n@highlight\nКомментарий: Провал российских спецслужб","[{'start': 47, 'end': 57}, {'start': 199, 'end': 209}, {'start': 223, 'end': 242}, {'start': 281, 'end': 291}, {'start': 371, 'end': 377}, {'start': 582, 'end': 592}, {'start': 607, 'end': 615}, {'start': 617, 'end': 631}, {'start': 633, 'end': 640}, {'start': 654, 'end': 660}, {'start': 709, 'end': 717}, {'start': 758, 'end': 768}, {'start': 789, 'end': 798}, {'start': 819, 'end': 829}]"
1,"О вторжении на Украину танковой колонны из РФ сообщил представитель СНБО Украины Лысенко. Москва опровергла эту информацию, назвав ее провокацией. На территорию восточной Украины из России вошла колонна из 32 танков, заявили власти в Киеве. Помимо этого, границу с Луганской областью пересекли 30 грузовиков с бойцами, 16 гаубиц и другая военная техника, сообщил представитель Совета национальной безопасности и обороны (СНБО) Украины Андрей Лысенко на брифинге в Киеве в пятницу, 7 ноября. По его словам, колонна движется в направлении города Красный луч. Переброска военного оборудования и российских наемников на линии фронта продолжается, резюмировал представитель СНБО.\n@highlight\nВ Германии за сутки выявлено более 100 новых заражений коронавирусом\n@highlight\nРыночные цены на нефть рухнули из-за провала переговоров ОПЕК+\n@highlight\nВ Италии за сутки произошел резкий скачок смертей от COVID-19","[{'start': 15, 'end': 22}, {'start': 43, 'end': 45}, {'start': 68, 'end': 72}, {'start': 73, 'end': 80}, {'start': 81, 'end': 88}, {'start': 90, 'end': 96}, {'start': 171, 'end': 178}, {'start': 182, 'end': 188}, {'start': 234, 'end': 239}, {'start': 265, 'end': 283}, {'start': 421, 'end': 425}, {'start': 427, 'end': 434}, {'start': 435, 'end': 449}, {'start': 464, 'end': 469}, {'start': 544, 'end': 555}, {'start': 669, 'end': 673}, {'start': 688, 'end': 696}, {'start': 823, 'end': 827}, {'start': 842, 'end': 848}]"
2,"Год назад Владимир Путин вновь стал президентом России. С тех пор протесты против его возвращения в Кремль поутихли. Но это внешнее спокойствие - обманчиво, считает Инго Маннтойфель. Настоящая стабильность возможна только при наличии правового государства и демократических институтов, которые способны поддерживать в равновесии различные общественные интересы. Ведь то, что может произойти в противном случае, не хочется даже себе представлять. @header Стабильность возможна только в правовом государстве Помня о русской традиции насильственных революционных переворотов, которые неизменно отбрасывали страну назад, хочется надеяться, что в Кремле осознают необходимость эволюционного развития и, следовательно, либеральных реформ.\n@highlight\nВ Германии за сутки выявлено более 100 новых заражений коронавирусом\n@highlight\nКомментарий: Россия накануне эпидемии - виноватые назначены заранее\n@highlight\nТуризм в эпоху коронавируса: куда поехать? И ехать ли вообще?","[{'start': 10, 'end': 24}, {'start': 48, 'end': 54}, {'start': 100, 'end': 106}, {'start': 165, 'end': 181}, {'start': 642, 'end': 648}, {'start': 746, 'end': 754}, {'start': 837, 'end': 843}]"
3,"Союз девяти ведущих технических университетов Германии TU9 не только 

In [26]:
questions_train.head()

,text_id,question,answers
0,0,"Кроме того, серьезным вызовом для России становится стремительно развивающийся Китай.Еще в понедельник @placeholder в рамках спора о системе противоракетной обороны пригрозил размещением дополнительных ракетных комплексов.","[{'start': 789, 'end': 798, 'text': 'Медведева'}]"
1,1,"Россия категорически опровергла сообщение @placeholder, назвав его провокацией.","[{'start': 81, 'end': 88, 'text': 'Лысенко'}, {'start': 435, 'end': 449, 'text': 'Андрей Лысенко'}]"
2,2,"@placeholder, руководитель отдела Восточной Европы и главный редактор русской редакции Deutsche Welle","[{'start': 165, 'end': 181, 'text': 'Инго Маннтойфель'}]"
3,3,"@placeholder позиционирует себя как союз независимых университетов, которые заинтересованы не столько в государственной поддержке, сколько в сотрудничестве с реальным сектором экономики, производством и бизнесом.","[{'start': 55, 'end': 58, 'text': 'TU9'}, {'start': 192, 'end': 195, 'text': 'TU9'}]"
4,4,Согласно указу президента @placeholder особые меры безопасности в местах проведения турнира действуют с 25 мая по 25 июля.,"[{'start': 546, 'end': 552, 'text': 'России'}]"


In [27]:
def fill_entities(text, entities):
    for entity in entities:
        entity['text'] = text[entity['start']:entity['end']]

for idx, row in texts_train.iterrows():
    fill_entities(row.text, row.entities)

for idx, row in texts_valid.iterrows():
    fill_entities(row.text, row.entities)

In [28]:
questions_train.iloc[0]

text_id                                                                                                                                                                                                                                  0
question    Кроме того, серьезным вызовом для России становится стремительно развивающийся Китай.Еще в понедельник @placeholder в рамках спора о системе противоракетной обороны пригрозил размещением дополнительных ракетных комплексов.
answers                                                                                                                                                                                  [{'start': 789, 'end': 798, 'text': 'Медведева'}]
Name: 0, dtype: object

In [29]:
texts_train.iloc[0]

text        Наблюдатели полагают, что подоплекой теракта в Домодедово является провал кавказской политики российского правительства, указывает немецкая печать. Немецкая печать продолжает комментировать теракт в Домодедово. Так, газета Süddeutsche Zeitung пишет:\nНу, конечно же, после взрыва в Домодедово вновь обнаруживается "кавказский след". Его обнаруживают почти всегда, когда в России взрывается бомба в метро, в поезде или на рынке. И неважно, что очевидцы нередко не в состоянии сказать, был ли преступник женщиной в чадре или же мужчиной, грозившим всех уничтожить.Среди жертв взрыва в Домодедово есть граждане Германии, Великобритании, Австрии. Ежедневно в Москве совершают посадку более 3 десятков самолетов из Германии.\n@highlight\nНемецкий менеджер аэропорта Домодедово отвергает обвинения Медведева\n@highlight\nТеракт в Домодедово: системные просчеты\n@highlight\nКомментарий: Провал российских спецслужб
entities                                                                       

# RSG Baseline

In [18]:
%%capture
!pip install jsonlines

In [19]:
import pickle
import codecs
import joblib
import RuCoS

vect = joblib.load("data/tfidf.pkl")

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.21.3 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [ ]:
train_path = "data/RuCoS/train.jsonl"
val_path = "data/RuCoS/val.jsonl"
test_path = "data/RuCoS/test.jsonl"

_, RuCoS_scores = RuCoS.eval_RuCoS(train_path, val_path, test_path, vect)


val_em = RuCoS_scores['val'][0]
val_f1 = RuCoS_scores['val'][1]

print(f"EM Score on Validation: {val_em}") # should be around 0.22

print(f"F1 Score on Validation: {val_f1}") # should be around 0.23

EM Score on Validation: 0.22964233865646033
F1 Score on Validation: 0.235315208305838


# Heruistics

## Удаление кандидатов + Random Choice

In [72]:
from sklearn.metrics.pairwise import cosine_similarity
import jsonlines
import numpy as np
from collections import Counter
import string
import re
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = [0]
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(dataset, predictions):
    f1 = exact_match = total = 0
    correct_ids = []
    for prediction, passage in zip(predictions, dataset):
        prediction = prediction["label"]
        for qa in passage['qas']:
            total += 1
            ground_truths = list(map(lambda x: x['text'], qa.get("answers", "")))

            _exact_match = metric_max_over_ground_truths(exact_match_score, prediction, ground_truths)
            if int(_exact_match) == 1:
                correct_ids.append(qa['idx'])
            exact_match += _exact_match

            f1 += metric_max_over_ground_truths(f1_score, prediction, ground_truths)

    exact_match = exact_match / total
    f1 = f1 / total
    return exact_match, f1


def eval_RuCoS(train_path, val_path, test_path, vect):
    test_score, test_pred = eval_part(test_path, vect)
    return None, {
        "train": eval_part(train_path, vect)[0],
        "val": eval_part(val_path, vect)[0],
        "test": test_score,
        "test_pred": test_pred
    }


def eval_part(path, vect):
    with jsonlines.open(path) as reader:
        lines = list(reader)
    preds = []
    for row in lines:
        pred = get_row_pred(row, vect)
        # print(row)
        # print('pred', pred)
        preds.append({
            "idx": row["idx"],
            "label": pred
        })
        # break
    return evaluate(lines, preds), preds


def get_row_pred(row, vect):
    # text = vect.transform([row["passage"]["text"].replace("\n@highlight\n", " ")])
    res = []
    words = [
        row["passage"]["text"][x["start"]: x["end"]]
        for x in row["passage"]["entities"]]
    # if len(row['qas']) > 1:
        # print(len(row['qas']))

    for line in row["qas"]:
        line_candidates = []
        _words = []
        for word in words:
            if word[:-1]  not in line['query']:
                _words.append(word)
        if len(_words) == 0:
            for word in words:
                line_candidates.append(line["query"].replace("@placeholder", word))
            pred_idx = np.random.choice(np.arange(1, len(line_candidates)),
                                size=1)[0]
            pred = np.array(words)[pred_idx]
        elif len(_words) == 1:
            pred = _words[0]
        else:
            for word in _words:
                line_candidates.append(line["query"].replace("@placeholder", word))
            pred_idx = np.random.choice(np.arange(1, len(line_candidates)),
                                        size=1)[0]
            pred = np.array(_words)[pred_idx]
        # cos = cosine_similarity(text, vect.transform(line_candidates))
        # pred = np.array(words)[cos.argsort()[0][-1]]
        res.append(pred)
    return " ".join(res)

In [73]:
em_metrics = []
f1_metrics = []

for i in range(3):
    _, RuCoS_scores = eval_RuCoS(train_path, val_path, test_path, 'No vect')
    em = RuCoS_scores['val'][0]
    f1 = RuCoS_scores['val'][1]
    em_metrics.append(em)
    f1_metrics.append(f1)

print(f"Random Choice")
print(f"Average EM score over 3 experiments: {np.array(em_metrics).mean()}")
print(f"Average F1 score over 3 experiments: {np.array(f1_metrics).mean()}")

Random Choice
Average EM score over 3 experiments: 0.24666754652237033
Average F1 score over 3 experiments: 0.24893841219563806


## Фильтрация + Count + Random Choice

In [101]:
from sklearn.metrics.pairwise import cosine_similarity
import jsonlines
import numpy as np
from collections import Counter
import string
import re
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return normalize_answer(prediction) == normalize_answer(ground_truth)


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = [0]
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(dataset, predictions):
    f1 = exact_match = total = 0
    correct_ids = []
    for prediction, passage in zip(predictions, dataset):
        prediction = prediction["label"]
        for qa in passage['qas']:
            total += 1
            ground_truths = list(map(lambda x: x['text'], qa.get("answers", "")))

            _exact_match = metric_max_over_ground_truths(exact_match_score, prediction, ground_truths)
            if int(_exact_match) == 1:
                correct_ids.append(qa['idx'])
            exact_match += _exact_match

            f1 += metric_max_over_ground_truths(f1_score, prediction, ground_truths)

    exact_match = exact_match / total
    f1 = f1 / total
    return exact_match, f1


def eval_RuCoS(train_path, val_path, test_path, vect):
    test_score, test_pred = eval_part(test_path, vect)
    return None, {
        "train": eval_part(train_path, vect)[0],
        "val": eval_part(val_path, vect)[0],
        "test": test_score,
        "test_pred": test_pred
    }


def eval_part(path, vect):
    with jsonlines.open(path) as reader:
        lines = list(reader)
    preds = []
    for row in lines:
        pred = get_row_pred(row, vect)
        preds.append({
            "idx": row["idx"],
            "label": pred
        })
    return evaluate(lines, preds), preds


def get_row_pred(row, vect):
    res = []
    words = [
        row["passage"]["text"][x["start"]: x["end"]]
        for x in row["passage"]["entities"]]
    text  = row['passage']['text'].split()
    for line in row["qas"]:
        line_candidates = []
        _words = []
        for word in words:
            if word[:-2]  not in line['query'] or text.count(words[:-2]) >= 2:
                _words.append(word)
        if len(_words) == 0:
            for word in words:
                line_candidates.append(line["query"].replace("@placeholder", word))
            pred_idx = np.random.choice(np.arange(1, len(line_candidates)),
                                size=1)[0]
            pred = np.array(words)[pred_idx]
        elif len(_words) == 1:
            pred = _words[0]
        else:
            for word in _words:
                line_candidates.append(line["query"].replace("@placeholder", word))
            pred_idx = np.random.choice(np.arange(1, len(line_candidates)),
                                        size=1)[0]
            pred = np.array(_words)[pred_idx]
        res.append(pred)
    return " ".join(res)

In [102]:
em_metrics = []
f1_metrics = []

for i in range(3):
    _, RuCoS_scores = eval_RuCoS(train_path, val_path, test_path, 'No vect')
    em = RuCoS_scores['val'][0]
    f1 = RuCoS_scores['val'][1]
    em_metrics.append(em)
    f1_metrics.append(f1)

print(f"Random Choice")
print(f"Average EM score over 3 experiments: {np.array(em_metrics).mean()}")
print(f"Average F1 score over 3 experiments: {np.array(f1_metrics).mean()}")

Random Choice
Average EM score over 3 experiments: 0.24794333729268403
Average F1 score over 3 experiments: 0.25025233004644387
